In [79]:
#level3

#imports
import cv2
import numpy as np
np.set_printoptions(threshold=np.inf)

class data_chunking_level_3():


    def __init__(self,height,width):
        self.height=height
        self.width=width
    
    def calculate_vector(self,gray,inner,start,end):
        vector = []
        if inner is False:
            for row in range(self.height):
                vector.append(int(np.mean(gray[row])))
        else:
            for column in range(self.width):
                average = 0
                for row in range(start,end):
                    average = average + gray[row,column]
                vector.append(int(average/(end-start)))
        return vector

    def primary_segmentation(self,gray,row_vector,line_pixel_count,primary):
        self.primary_horizontal_skip_pixel=primary[0]
        self.primary_horizontal_threshold=primary[1]
        self.primary_horizontal_line_threshold=primary[2]
        counter=0
        prev=0
        horizontal_chunks_index = []
        row_flag=False
        done=False
        for row in range(self.height):
            counter=0
            for column in range(self.width):
                if gray[row][column]<=250 and prev==gray[row][column]:
                    counter=counter+1
                else:
                    counter=0
                if counter>=int(line_pixel_count):
                    if row not in horizontal_chunks_index:
                        horizontal_chunks_index.append(row)
                    counter=0
                    done=True
                prev=gray[row][column]
            if not done:
                if row<= self.height-self.primary_horizontal_skip_pixel:
                    value=0
                    for horizontal_pixel in range(row,row+self.primary_horizontal_skip_pixel):
                        value=value+row_vector[horizontal_pixel]
                    value=int(value/self.primary_horizontal_skip_pixel)
                    if value >= self.primary_horizontal_threshold or row_vector[row]<=self.primary_horizontal_line_threshold:
                        if not row_flag:
                            row_flag=True
                        if row not in horizontal_chunks_index:
                            horizontal_chunks_index.append(row)
                    else:
                        row_flag=False
            else:
                done=False
        return horizontal_chunks_index
    
    def accumulate_whitespaces(self,horizontal_chunks_index):
        deviation_index=0
        while deviation_index <= len(horizontal_chunks_index)-2:
            if horizontal_chunks_index[deviation_index] - horizontal_chunks_index[deviation_index+1] == -1:
                del horizontal_chunks_index[deviation_index]
            else:
                deviation_index= deviation_index+1
                continue
        return horizontal_chunks_index

#     def print_primary_rows(self,horizontal_chunks_index, resized_image):
#         for index in range(len(horizontal_chunks_index)):
#             #resized_image=self.draw_on_image(resized_image,[[0,horizontal_chunks_index[index]],[self.width,horizontal_chunks_index[index]]])
#         return resized_image
    
    def print_secondary_columns(self,row_vector,column_vector,resized_image,start_index,end_index,secondary_vertical,inner_dict):
        self.secondary_vertical_skip_pixel=secondary_vertical[0]
        self.secondary_vertical_threshold=secondary_vertical[1]
        self.secondary_vertical_line_threshold=secondary_vertical[2]
        column_flag=False
        whitespace_start_value=0
        for column in range(self.width):
            if column <= self.width-self.secondary_vertical_skip_pixel:
                value=0
                for vertical_pixel in range(column,column+self.secondary_vertical_skip_pixel):
                    value=value+column_vector[vertical_pixel]
                value=int(value/self.secondary_vertical_skip_pixel)
                if value >=self.secondary_vertical_threshold or column_vector[column]<=self.secondary_vertical_line_threshold:
                    if not column_flag:
                        column_flag=True
                        whitespace_start_value=column
                else:
                    if column_flag:
                        inner_dict["columns"].append(int((whitespace_start_value+column)/2))
                    column_flag=False
                    
        return resized_image,inner_dict
    
    def print_secondary_rows(self,row_vector,column_vector,resized_image,start_index,end_index,secondary_horizontal,inner_dict):
        self.secondary_horizontal_skip_pixel=secondary_horizontal[0]
        self.secondary_horizontal_threshold=secondary_horizontal[1]
        self.secondary_horizontal_line_threshold=secondary_horizontal[2]
        row_flag=False
        whitespace_start_value=0 
        for row in range(start_index,end_index):
            if row<= end_index -self.secondary_horizontal_skip_pixel:
                value=0
                for horizontal_pixel in range(row,row+self.secondary_horizontal_skip_pixel):
                    value=value+row_vector[horizontal_pixel]
                value=int(value/self.secondary_horizontal_skip_pixel)
                if value >= self.secondary_horizontal_threshold or row_vector[row]<=self.secondary_horizontal_line_threshold:
                    if not row_flag:
                        #resized_image = self.draw_on_image(resized_image,[[0,row],[width,row]])
                        row_flag=True
                        whitespace_start_value=row
                else:
                    if row_flag:
                        inner_dict["rows"].append(int((whitespace_start_value+row)/2))
                    row_flag=False
        return resized_image,inner_dict
    

In [80]:
#level 2

#imports
import cv2
import numpy as np
np.set_printoptions(threshold=np.inf)

class data_chunking_level_2():
    height=1000
    width=800

    skip_color_start=256
    skip_color_end=-1

    image='Report_Images/888.png'
    
    def __init__(self,height,width,skip_color_start,skip_color_end):
        self.height=height
        self.width=width
        self.skip_color_start=skip_color_start
        self.skip_color_end=skip_color_end
        self.image=image
        self.d3=data_chunking_level_3(height,width)
    
    
    def read_and_resize_image(self,image):
        img = cv2.imread(image)
        resized_image = cv2.resize(img, (self.width,self.height))
        gray = cv2.cvtColor(resized_image,cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray,50,150,apertureSize = 3)
        for row in range(self.height):
            for column in range(self.width):
                if gray[row,column] >=self.skip_color_start and gray[row,column] <= self.skip_color_end:
                    gray[row,column] = 255
        return gray,resized_image
    
    
    def draw_segmented_rows(self,gray,primary,resized_image,line_pixel_count):
        row_vector=self.d3.calculate_vector(gray,False,0,0)
        horizontal_chunks_index=self.d3.primary_segmentation(gray,row_vector,line_pixel_count,primary)
        horizontal_chunks_index = self.d3.accumulate_whitespaces(horizontal_chunks_index)
#         resized_image = self.d3.print_primary_rows(horizontal_chunks_index,resized_image)
        return horizontal_chunks_index,row_vector,resized_image

    def draw_table_in_chunks(self,gray,secondary_vertical,secondary_horizontal,horizontal_chunks_index,row_vector,resized_image):
        height_index = 0
        print(horizontal_chunks_index)
        data = []
        while height_index <= len(horizontal_chunks_index) - 2:
            inner_dict = {"start":0,"end":0,"columns":[],"rows":[]}
            start=height_index
            end=height_index+1
            start_index=horizontal_chunks_index[start]
            end_index=horizontal_chunks_index[end]
            inner_dict["start"] = start_index
            inner_dict["end"] = end_index
            column_vector=self.d3.calculate_vector(gray,True,start_index,end_index)
            resized_image,inner_dict=self.d3.print_secondary_columns(row_vector,column_vector,resized_image,start_index,end_index,secondary_vertical,inner_dict)
            resized_image,inner_dict=self.d3.print_secondary_rows(row_vector,column_vector,resized_image,start_index,end_index,secondary_horizontal,inner_dict) 
            height_index = end
            data.append(inner_dict)
        return resized_image,data
    
    def display_image(self,image_name,window_name):
        cv2.imshow(window_name,image_name)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [81]:
#level1
class data_chunking_level_1():

    
    def __init__(self,height,width,skip_color_start,skip_color_end):
        self.d2=data_chunking_level_2(height,width,skip_color_start,skip_color_end)
    
    def draw_row_chunks_on_image_file(self,image,primary,line_pixel_count):
        gray,resized_image=self.d2.read_and_resize_image(image)
        horizontal_chunks_index,row_vector,resized_image = self.d2.draw_segmented_rows(gray,primary,resized_image,line_pixel_count)
        return horizontal_chunks_index
    
    def draw_table_on_image_file(self,image,primary,secondary_vertical,secondary_horizontal,line_pixel_count):
        data_dict = {"height":0,"width":0,"chunk":[]}
        gray,resized_image=self.d2.read_and_resize_image(image)
        horizontal_chunks_index,row_vector,resized_image = self.d2.draw_segmented_rows(gray,primary,resized_image,line_pixel_count)
        resized_image,data = self.d2.draw_table_in_chunks(gray,secondary_vertical,secondary_horizontal,horizontal_chunks_index,row_vector,resized_image)
        data_dict["height"] = height
        data_dict["width"] = width 
        data_dict["chunk"] = data
        return data_dict
        

In [82]:
class image_interpreter():

    def __init__(self,red,green,blue):
        self.red = red
        self.blue = blue
        self.green = green
              
    def draw_on_image(self,image,cordinates_list=[[]]):
        image = cv2.line(image,(cordinates_list[0][0],cordinates_list[0][1]),(cordinates_list[1][0],cordinates_list[1][1]),(self.blue,self.green,self.red),1)    
        return image
    
    def display_image(self,image_name,window_name):
        cv2.imshow("view",image_name)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def draw_primary_row_chunks(self,image,data_dict):
        img = cv2.imread(image)
        resized_image = cv2.resize(img, (data_dict["width"],data_dict["height"]))
        for inner_dict in data_dict["chunk"]:
            start = inner_dict['start']
            image=self.draw_on_image(resized_image,[[0,start],[data_dict["width"],start]])
        self.display_image(image,"view")
    
    def draw_table(self,image,data_dict):
        img = cv2.imread(image)
        resized_image = cv2.resize(img, (data_dict["width"],data_dict["height"]))
        for inner_dict in data_dict["chunk"]:
            start = inner_dict['start']
            end = inner_dict['end']
            column_list = inner_dict['columns']
            row_list = inner_dict['rows']                                                   
            image=self.draw_on_image(resized_image,[[0,start],[data_dict["width"],start]])
                                                               
            for column in column_list:
                image=self.draw_on_image(image,[[column,start],[column,end]])
             
            for row in row_list:
                 image=self.draw_on_image(image,[[0,row],[data_dict["width"],row]])                                              
        
        self.display_image(image,"view")                                                   
               
    
    

In [87]:
#crop operation
import cv2
from google.cloud.vision.client import Client



def crop_image(image_location,my_dict):
    out=get_coordinates(my_dict)
    image = cv2.imread(image_location)
    resized_image = cv2.resize(image,(my_dict['width'],my_dict['height']))
    for primary_row in out:
        for secondary_column in primary_row:
            for secondary_row in secondary_column:
                crop = resized_image[secondary_row['y0']:secondary_row['y1'],secondary_row['x0']:secondary_row['x1']].copy()
                gray_crop = cv2.cvtColor(crop,cv2.COLOR_BGR2GRAY)
                avg=np.mean(np.mean(gray_crop))
                if avg<255.0:
                    vision_client = vision.Client()
                    content = crop.read()
                    image = vision_client.image(content=content)
                    want = image.detect_full_text()
                    print(want.text)
                    cv2.imshow("c",crop)
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()
    return out

def get_coordinates(my_dict):
    main_list = []
    for inner_dict in my_dict["chunk"]:
        inner_list = []
        if len(inner_dict["columns"]) > 0:
            column_list = inner_dict["columns"]
            x0 = 0
            for column in range(len(column_list)+1):
                if column == len(column_list):
                    x1 = my_dict['width']
                    get_rows(inner_dict,inner_list,x0,x1)
                    break

                else:
                    x1 = column_list[column]
                    if x1 != x0:
                        get_rows(inner_dict,inner_list,x0,x1)
                    x0 = x1 
        else:
            x0 = 0
            x1 = 600
            val={'x0': x0,'x1':x1,'y0':inner_dict['start'],'y1':inner_dict['end'],'text':""}
            if val not in inner_list:
                inner_list.append(val)
        main_list.append(inner_list)
        
    return main_list   

def get_rows(inner_dict,inner_list,x0,x1):
    
    if len(inner_dict["rows"]) > 0:
        row_list = inner_dict["rows"]
        y0 = inner_dict['start']
        deep_inner_list=[]
        for row in range(len(row_list)+1):
            if row == len(row_list):
                y1 = inner_dict['end']
                if deep_inner_list not in inner_list:
                    inner_list.append(deep_inner_list)
                val={'x0': x0,'x1':x1,'y0':y0,'y1':y1,'text':""}
                if val not in deep_inner_list:
                    deep_inner_list.append(val)
                break

            else:
                y1 = row_list[row]
                if y1 != y0:
                    if deep_inner_list not in inner_list:
                        inner_list.append(deep_inner_list)
                    val={'x0': x0,'x1':x1,'y0':y0,'y1':y1,'text':""}
                    if val not in deep_inner_list:
                        deep_inner_list.append(val)
                y0 = y1  
    else :
        deep_inner_list=[]
        if deep_inner_list not in inner_list:
            inner_list.append(deep_inner_list)
        val={'x0': x0,'x1':x1,'y0':inner_dict['start'],'y1':inner_dict['end']}
        if val not in deep_inner_list:
            deep_inner_list.append(val)


FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\setuptools-27.2.0-py3.6.egg'

In [ ]:
#client

height=800
width=600

skip_color_start=175
skip_color_end=225

blue=0
red=0
green =0

image='../Report_Images/888.png'

chunking = data_chunking_level_1(height,width,skip_color_start,skip_color_end)
data_dict = chunking.draw_table_on_image_file(image,[10,254,165],[7,253,-1],[6,254,165],200)

print(data_dict)

interpret = image_interpreter(red,green,blue)
# interpret.draw_primary_row_chunks(image,data_dict)
interpret.draw_table(image,data_dict)

out=crop_image(image,data_dict)
